<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/banner.png)

# <h1><center>Tarea Tutorial - Semana 1 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/PruebaBanner.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>Aproximación de Funciones</h1></center>

En este tutorial aprenderás acerca de las ventajas que tiene utilizar aproximación de funciones en problemas de aprendizaje por refuerzo profundo. Ilustraremos el proceso con el problema de <a href="https://gymnasium.farama.org/environments/classic_control/cart_pole/">Cart Pole</a>, incluido en las librerías de Gym, e implementaremos redes nuronales con el framework <a href="https://github.com/inarikami/keras-rl2/blob/master/examples/dqn_cartpole.py">keras-rl2</a>, que ya incluye varios algoritmos de aprendizaje por refuerzo con tensorflow. El objetivo principal será comparar el desempeño de distintos algoritmos en su versión tabular y su versión con redes neuronales. Este notebook tutorial se divide en las siguientes secciones:

# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#objetivos-de-aprendizaje)  
2. [Marco Teórico](#marco-teorico)  
3. [Instalación de Librerías](#instalacion-de-librerias)  
4. [Familiarización con el Entorno de Gym](#familiarizacion-con-el-entorno-de-gym)  
5. [Métodos Tabulares](#metodos-tabulares)  
6. [Métodos con Redes Neuronales](#metodos-con-redes-neuronales)  
7. [Comparación](#comparacion)  
8. [Conclusiones](#conclusiones)  

## Objetivos de Aprendizaje  
Contenido de la sección 1.  

## Marco Teórico  
Contenido de la sección 2.  

## Instalación de Librerías  
Contenido de la sección 3.  

## Familiarización con el Entorno de Gym  
Contenido de la sección 4.  

## Métodos Tabulares  
Contenido de la sección 5.  

## Métodos con Redes Neuronales  
Contenido de la sección 6.  

## Comparación  
Contenido de la sección 7.  

## Conclusiones  
Contenido de la sección 8.  

